In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy
# 1923 earliest year


In [2]:
# read in new_old merged data
new_path = input("Enter the file path: ")

new = pd.read_parquet(new_path)


In [3]:
# read in usda clean data
usda_path = input("Enter the file path: ")

# read in the old species data in json format
usda = pd.read_parquet(usda_path)


In [4]:
# rename usda column species to scientific_name
usda.rename(columns={"species": "scientific_name"}, inplace=True)


In [5]:
inner = pd.merge(new, usda, how="inner", on=[
                 "scientific_name"], indicator=True)


In [6]:
inner["combined_id"] = inner["unique_id_x"].apply(lambda x: x.tolist()) + inner[
    "unique_id_y"
].apply(lambda x: x.tolist())


In [7]:
inner["combined_id"] = inner["combined_id"].apply(set).apply(list)


In [8]:
check_combined_id = inner[["unique_id_x", "unique_id_y", "combined_id"]]


In [9]:
inner = inner.drop(["unique_id_x", "unique_id_y"], axis=1)

inner.rename(columns={"combined_id": "unique_id"}, inplace=True)

del check_combined_id


In [10]:
inner


,kingdom,phylum,class,order,family_x,genus,scientific_name,common_name_x,species_type_x,rights,species_type_y,common_name_y,family_y,_merge,unique_id
0,Plantae,Tracheophyta,Magnoliopsida,Caryophyllales,Plumbaginaceae,Armeria,Armeria maritima,thrift seapink,wildflower,"[{'license': 'CC_BY_4_0', 'rightsHolder': None...",wildflower,thrift seapink,Plumbaginaceae,both,"[659, 847, 352, 393, 397, 846, 411, 377, 339, ..."
1,Plantae,Tracheophyta,Magnoliopsida,Poales,Cyperaceae,Bolboschoenus,Bolboschoenus maritimus,cosmopolitan bulrush,graminoid,"[{'license': 'CC_BY_4_0', 'rightsHolder': None...",graminoid,cosmopolitan bulrush,Cyperaceae,both,"[323, 416, 417, 326, 421, 312, 346, 422, 388, ..."
2,Plantae,Tracheophyta,Magnoliopsida,Ceratophyllales,Ceratophyllaceae,Ceratophyllum,Ceratophyllum demersum,coon's tail,wildflower,"[{'license': 'CC0_1_0', 'rightsHolder': None},...",wildflower,coon's tail,Ceratophyllaceae,both,"[323, 416, 417, 326, 421, 312, 346, 422, 388, ..."
3,Plantae,Tracheophyta,Magnoliopsida,Poales,Poaceae,Phragmites,Phragmites australis,common reed,graminoid,"[{'license': 'CC0_1_0', 'rightsHolder': None},...",graminoid,common reed,Poaceae,both,"[323, 416, 417, 632, 326, 705, 649, 421, 312, ..."
4,Plantae,Tracheophyta,Polypodiopsida,Polypodiales,Dennstaedtiaceae,Pteridium,Pteridium aquilinum,western brackenfern,other_plants,"[{'license': 'CC_BY_4_0', 'rightsHolder': 'EDP...",wildflower,western brackenfern,Dennstaedtiaceae,both,"[323, 393, 416, 417, 326, 705, 649, 421, 312, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Plantae,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xanthium,Xanthium strumarium,rough cocklebur,wildflower,"[{'license': 'CC_BY_4_0', 'rightsHolder': 'EDP...",wildflower,rough cocklebur,Asteraceae,both,"[323, 416, 417, 326, 924, 421, 312, 346, 422, ..."
420,Plantae,Tracheophyta,Liliopsida,Asparagales,Asparagaceae,Yucca,Yucca aloifolia,aloe yucca,tree_shrub,"[{'license': 'CC_BY_4_0', 'rightsHolder': None}]",tree_shrub,aloe yucca,Agavaceae,both,"[567, 314, 316, 424, 776, 390, 345, 313, 315, ..."
421,Plantae,None,Magnoliopsida,Alismatales,Ruppiaceae,Ruppia,Ruppia cirrhosa,Spiral Tasselweed,wildflower,"[{'license': None, 'rightsHolder': None}]",wildflower,None,RUPPIACEAE,both,"[646, 393, 857, 722]"
422,Plantae,None,Magnoliopsida,Alismatales,Ruppiaceae,Ruppia,Ruppia maritima,widgeongrass,wildflower,"[{'license': None, 'rightsHolder': None}]",wildflower,widgeongrass,Ruppiaceae,both,"[323, 393, 421, 312, 679, 346, 422, 424, 333, ..."


In [11]:
# keep columns with less missing values


family_x_missing = inner["family_x"].isna().sum()

family_y_missing = inner["family_y"].isna().sum()

# drop the family column from inner that contains more missing values

if family_x_missing > family_y_missing:
    inner = inner.drop(["family_x"], axis=1)
    inner.rename(columns={"family_y": "family"}, inplace=True)
else:
    inner = inner.drop(["family_y"], axis=1)
    inner.rename(columns={"family_x": "family"}, inplace=True)

In [12]:
del family_x_missing, family_y_missing


In [14]:
# drop species_type_y and rename species_type_x to species_type

inner = inner.drop(["species_type_y"], axis=1)

inner.rename(columns={"species_type_x": "species_type"}, inplace=True)


In [15]:
common_name_x_missing = inner["common_name_x"].isna().sum()

common_name_y_missing = inner["common_name_y"].isna().sum()


In [16]:
if common_name_x_missing > common_name_y_missing:
    inner = inner.drop(["common_name_x"], axis=1)
    inner.rename(columns={"common_name_y": "common_name"}, inplace=True)
else:
    inner = inner.drop(["common_name_y"], axis=1)
    inner.rename(columns={"common_name_x": "common_name"}, inplace=True)

del common_name_x_missing, common_name_y_missing


In [17]:
common_name_missing = inner["common_name"].isna().sum()


In [18]:
# drop _merge column

inner = inner.drop(["_merge"], axis=1)

In [19]:
inner = inner[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

del common_name_missing

In [20]:
outer = pd.merge(new, usda, how="outer", on=[
                 "scientific_name"], indicator=True)


In [21]:
outer


,kingdom,phylum,class,order,family_x,genus,scientific_name,common_name_x,species_type_x,unique_id_x,rights,species_type_y,common_name_y,family_y,unique_id_y,_merge
0,Animalia,Chordata,Actinopteri,Cypriniformes,Cyprinidae,Abramis,Abramis brama,Common Bream,fish,"[705, 845, 632, 846]","[{'license': 'CC_BY_4_0', 'rightsHolder': 'Ume...",NaN,NaN,NaN,NaN,left_only
1,Animalia,Echinodermata,Asteroidea,Valvatida,Acanthasteridae,Acanthaster,Acanthaster planci,Long-spined Crown-of-Thorns,worm,"[927, 928, 954]","[{'license': 'CC0_1_0', 'rightsHolder': None},...",NaN,NaN,NaN,NaN,left_only
2,Animalia,Chordata,Actinopteri,Scombriformes,Scombridae,Acanthocybium,Acanthocybium solandri,Wahoo,fish,"[850, 954, 926, 1048]","[{'license': 'CC0_1_0', 'rightsHolder': None},...",NaN,NaN,NaN,NaN,left_only
3,Plantae,Rhodophyta,Florideophyceae,Ceramiales,Rhodomelaceae,Acanthophora,Acanthophora spicifera,prickly seaweed,algae,"[198, 247]","[{'license': 'CC0_1_0', 'rightsHolder': None},...",NaN,NaN,NaN,NaN,left_only
4,Animalia,Chordata,Actinopteri,Acanthuriformes,Acanthuridae,Acanthurus,Acanthurus blochii,Dark Surgeonfish,fish,"[628, 606, 605, 616, 926, 935, 955, 610, 627, ...","[{'license': 'CC0_1_0', 'rightsHolder': None},...",NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26573,NaN,NaN,NaN,NaN,NaN,NaN,Petitia domingensis,NaN,NaN,NaN,NaN,tree_shrub,bastard stopper,Verbenaceae,[567],right_only
26574,NaN,NaN,NaN,NaN,NaN,NaN,Pisonia floridana,NaN,NaN,NaN,NaN,tree_shrub,Rock Key devil's-claws,Nyctaginaceae,[567],right_only
26575,NaN,NaN,NaN,NaN,NaN,NaN,Castilleja fraterna,NaN,NaN,NaN,NaN,wildflower,fraternal Indian paintbrush,Scrophulariaceae,[333],right_only
26576,NaN,NaN,NaN,NaN,NaN,NaN,Cirsium peckii,NaN,NaN,NaN,NaN,wildflower,Steen Mountain thistle,Asteraceae,[333],right_only


In [22]:
left = outer.loc[
    outer._merge == "left_only",
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family_x",
        "genus",
        "scientific_name",
        "common_name_x",
        "species_type_x",
        "unique_id_x",
        "rights",
    ],
]


In [23]:
left.rename(
    columns={
        "family_x": "family",
        "unique_id_x": "unique_id",
        "common_name_x": "common_name",
        "species_type_x": "species_type",
    },
    inplace=True,
)


In [24]:
left = left[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [25]:
right = outer.loc[
    outer._merge == "right_only",
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family_y",
        "genus",
        "scientific_name",
        "common_name_y",
        "species_type_y",
        "unique_id_y",
        "rights",
    ],
]


In [26]:
right.rename(
    columns={
        "family_y": "family",
        "unique_id_y": "unique_id",
        "common_name_y": "common_name",
        "species_type_y": "species_type",
    },
    inplace=True,
)


In [27]:
right = right[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [28]:
frames = [inner, left, right]

# concatenate dataframes
final = pd.concat(frames)

# reset index
final.reset_index(drop=True, inplace=True)


In [29]:
# find unique values in scientific_name column of final
species_unique = final["scientific_name"].unique()


In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_merge_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
final.to_parquet(df_merge_path)
